In [8]:
import tensorflow as tf
from flask import Flask, request
import numpy as np
import mediapipe as mp
import cv2

In [6]:
model = tf.keras.models.load_model('my_model.keras')

In [7]:
actions = np.array(['hola', 'ok', 'a'])

In [10]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [9]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [11]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                              mp_drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),
                              mp_drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1))
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [20]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    right = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(63)
    left = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(63)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
    return np.concatenate([pose, face, right, left])

In [17]:
holistic =  mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)

In [34]:
from flask import jsonify, render_template
from flask_socketio import SocketIO, send

In [38]:
app = Flask(__name__)
socketio = SocketIO(app)
socketio.init_app(app, cors_allowed_origins="*")

predictions = []
threshold = 0.5
sequence = []

@socketio.on("connect")
def on_connect():
    print("Cliente conectado")

@app.route("/")
def index():
    return render_template("index.html")

@socketio.on("message")
def on_message(data):
    #data = request.get_json()
    frames = data["frames"]
    for frame in frames:
        image, result = mediapipe_detection(frame, holistic)
        keypoints = extract_keypoints(result)
        sequence.append(keypoints)
        #print(sequence)
    sequence = sequence[-30:]
    
    res = model.predict(np.expand_dims(sequence, axis=0))[0]    
    prediction = actions[np.argmax(res)]
    print(prediction)        

    

    socketio.emit("prediction", {"prediction": prediction})

if __name__ == "__main__":
    socketio.run(app, host="localhost", port=5000, allow_unsafe_werkzeug=True)

Werkzeug appears to be used in a production deployment. Consider switching to a production web server instead.


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:5000
Press CTRL+C to quit


127.0.0.1 - - [03/Apr/2024 20:32:50] "GET /socket.io HTTP/1.1" 404 -
127.0.0.1 - - [03/Apr/2024 20:32:55] "GET / HTTP/1.1" 400 -
